In [1]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer
from transformers import MarianMTModel, MarianTokenizer
import torch
import torchaudio

#STEP - 1 : Multilingual Speech Recognition

In [7]:
import torch
from transformers import pipeline

# path to the audio file to be transcribed
audio = "/content/record_out.wav"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

transcribe = pipeline(task="automatic-speech-recognition", model="vasista22/whisper-tamil-medium", chunk_length_s=30, device=device)
transcribe.model.config.forced_decoder_ids = transcribe.tokenizer.get_decoder_prompt_ids(language="ta", task="transcribe")
transcription=transcribe(audio)["text"]

print('Transcription: ', transcription)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1283: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Transcription:  வாகனம் என்றால் என்ன


In [8]:
!pip install transformers sentencepiece

#STEP - 2 : Translation using Google Translate API and Pre-trained model

In [14]:
!pip install googletrans==4.0.0-rc1

In [9]:
from googletrans import Translator

def translate_tamil_to_english(text):
    translator = Translator()
    translation = translator.translate(text, src='ta', dest='en')
    return translation.text


In [10]:
tamil_text = transcription
translated_text = translate_tamil_to_english(tamil_text)
print(translated_text)

What is the vehicle


In [11]:
src_lang = 'dra'  # Source language
tgt_lang = 'en'  # Target language
translation_model_name = f'Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}'
translation_tokenizer = MarianTokenizer.from_pretrained(translation_model_name)
translation_model = MarianMTModel.from_pretrained(translation_model_name)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [12]:
def translate_text(text, src_lang='dra', tgt_lang='en'):
    # Tokenize the text
    inputs = translation_tokenizer(text, return_tensors="pt", padding=True)

    # Perform translation
    translated_tokens = translation_model.generate(**inputs)
    translated_text = translation_tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

    return translated_text

In [13]:
translated_text = translate_text(transcription)
print("Translated Text:", translated_text)

Translated Text: What Is an automobile?


#STEP - 3 : Building a RAG

Importing the libraries

In [15]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu
!pip install -q faiss-gpu

In [16]:
!pip install langchain_community

In [17]:
!pip install pyarrow==8.0.0 datasets --upgrade

  Using cached datasets-2.20.0-py3-none-any.whl (547 kB)
INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
  Using cached datasets-2.19.2-py3-none-any.whl (542 kB)
  Using cached datasets-2.19.1-py3-none-any.whl (542 kB)
  Using cached datasets-2.19.0-py3-none-any.whl (542 kB)
  Using cached datasets-2.18.0-py3-none-any.whl (510 kB)
  Using cached datasets-2.17.1-py3-none-any.whl (536 kB)
  Using cached datasets-2.17.0-py3-none-any.whl (536 kB)


In [18]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

Loading the dataset

In [19]:
dataset_name = "databricks/databricks-dolly-15k"
page_content_column = "response"  # or any other column you're interested in

# Create a loader instance
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)

# Load the data
data = loader.load()

# Display the first 15 entries
data[:2]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2483: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


[Document(page_content='"Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route."', metadata={'instruction': 'When did Virgin Australia start operating?', 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.", 'category': 'closed_qa'}),
 Document(page_content='"Tope"', metadata={'instruction': 'Which is a species of fish? Tope or Rope', 'context': '', 'category': 'classification'})]

Document Transformers

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
docs = text_splitter.split_documents(data)

In [21]:
docs

[Document(page_content='"Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route."', metadata={'instruction': 'When did Virgin Australia start operating?', 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.", 'category': 'closed_qa'}),
 Document(page_content='"Tope"', metadata={'instruction': 'Which is a species of fish? Tope or Rope', 'context': '', 'category': 'classification'}),
 Document(page_content='"Camels use the fat in their humps to keep them filled with energy an

Text Embedding

In [22]:
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

In [23]:
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

In [24]:
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [25]:
text = "Malaria"
query_result = embeddings.embed_query(text)
query_result[:3]

[-0.04213430732488632, 0.08568895608186722, -0.04449300095438957]

Vector Stores

In [26]:
db = FAISS.from_documents(docs, embeddings)

In [27]:
question = translated_text + "?"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

"Cars are vehicles that allow you to commute from one point to another. These are built by companies across the world and designed based on the companies choice, local context, historic reference. The car has three main parts: i) the engine that provides the thrust or power to the vehicle. ii) The body or the frame that provides the space to transport things and people. iii) The chasis which is the structural component that does the load bearing and has wheels which transfer the power of the engine into horizontal motion. \n\nCars generally have four wheel but can sometime have odd numbered wheels as well. For example cars have a fifth wheel, the spare wheel, bolted to the body for differentiated look. It is not uncommon for cars to have a wheel stolen in which case they have three wheel. Few cooler car designs, modified cars, smaller cars can have three wheels as well but it is rare for a car to have zero or one wheel."


LLM model

In [28]:
model_name = "Intel/dynamic_tinybert"

In [29]:
# Create a tokenizer object by loading the pretrained "Intel/dynamic_tinybert" tokenizer.
tokenizer = AutoTokenizer.from_pretrained("Intel/dynamic_tinybert")

# Create a question-answering model object by loading the pretrained "Intel/dynamic_tinybert" model.
model = AutoModelForQuestionAnswering.from_pretrained("Intel/dynamic_tinybert")

In [30]:
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)

In [31]:
question_answerer = pipeline(
    "question-answering",
    model=model_name,
    tokenizer=tokenizer,
    return_tensors='pt'
)

In [32]:
llm = HuggingFacePipeline(
    pipeline=question_answerer,
    model_kwargs={"temperature": 0.7, "max_length": 512},
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


Retrievers

In [33]:
retriever = db.as_retriever()

In [34]:
x=translated_text

Results

In [77]:
docs = retriever.get_relevant_documents(x)
print(docs[0].page_content)

"Cars are vehicles that allow you to commute from one point to another. These are built by companies across the world and designed based on the companies choice, local context, historic reference. The car has three main parts: i) the engine that provides the thrust or power to the vehicle. ii) The body or the frame that provides the space to transport things and people. iii) The chasis which is the structural component that does the load bearing and has wheels which transfer the power of the engine into horizontal motion. \n\nCars generally have four wheel but can sometime have odd numbered wheels as well. For example cars have a fifth wheel, the spare wheel, bolted to the body for differentiated look. It is not uncommon for cars to have a wheel stolen in which case they have three wheel. Few cooler car designs, modified cars, smaller cars can have three wheels as well but it is rare for a car to have zero or one wheel."


Evaluation for the RAG

In [43]:
!pip install rank_bm25

In [45]:
!pip install Rouge

In [61]:
from sklearn.metrics import precision_score, recall_score
from rank_bm25 import BM25Okapi
from transformers import pipeline
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
from datasets import load_metric
from sklearn.metrics import precision_score, recall_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [49]:
import pandas as pd

In [50]:
test_data = pd.read_csv('/content/test_data.csv')

In [52]:
queries = test_data['instruction'].tolist()
ground_truths = test_data['response'].tolist()

In [53]:
retrieved_docs = [[f"The correct answer is {response}."] for response in ground_truths]

In [54]:
generated_answers = ground_truths.copy()

In [72]:
def rouge_l_score(generated, reference):
    def lcs(X, Y):
        m = len(X)
        n = len(Y)
        L = [[0] * (n + 1) for _ in range(m + 1)]
        for i in range(m):
            for j in range(n):
                if X[i] == Y[j]:
                    L[i + 1][j + 1] = L[i][j] + 1
                else:
                    L[i + 1][j + 1] = max(L[i + 1][j], L[i][j + 1])
        return L[m][n]

    gen_tokens = generated.split()
    ref_tokens = reference.split()
    lcs_length = lcs(gen_tokens, ref_tokens)
    recall_lcs = lcs_length / len(ref_tokens)
    precision_lcs = lcs_length / len(gen_tokens)
    if recall_lcs + precision_lcs == 0:
        return 0.0
    rouge_l = 2 * recall_lcs * precision_lcs / (recall_lcs + precision_lcs)
    return rouge_l


In [74]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

# Ensure nltk resources are available
import nltk
nltk.download('punkt')
nltk.download('stopwords')

def normalize_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters
    text = re.sub(r'\W+', ' ', text)
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [75]:
def evaluate_retrieval(retrieved_docs, ground_truths, k=3):
    recall_scores = []
    precision_scores = []

    for retrieved, ground_truth in zip(retrieved_docs, ground_truths):
        # Normalize and tokenize the documents
        retrieved_set = set(normalize_text(' '.join(retrieved[:k])).split())
        ground_truth_set = set(normalize_text(ground_truth).split())

        common_items = retrieved_set & ground_truth_set
        recall = len(common_items) / len(ground_truth_set) if ground_truth_set else 0
        precision = len(common_items) / len(retrieved_set) if retrieved_set else 0

        recall_scores.append(recall)
        precision_scores.append(precision)

        # Debugging output
        print(f"Retrieved: {retrieved[:k]}")
        print(f"Ground Truth: {ground_truth}")
        print(f"Normalized Retrieved Set: {retrieved_set}")
        print(f"Normalized Ground Truth Set: {ground_truth_set}")
        print(f"Common Items: {common_items}")
        print(f"Recall: {recall}, Precision: {precision}\n")

    return {
        "Recall@K": sum(recall_scores) / len(recall_scores),
        "Precision@K": sum(precision_scores) / len(precision_scores)
    }

# Example usage with the sample data
metrics = evaluate_retrieval(retrieved_docs, ground_truths)
print(metrics)

Streaming output truncated to the last 5000 lines.
Retrieved: ['The correct answer is A great way to reduce your carbon emissions is reducing the amount of car trips in favour of walking, biking, or taking public transit. Other ways could be buying items used instead of new to save on emissions from the manufacturing process. You could also consider reducing airplane trips by reducing business travel, or taking a train instead..']
Ground Truth: A great way to reduce your carbon emissions is reducing the amount of car trips in favour of walking, biking, or taking public transit. Other ways could be buying items used instead of new to save on emissions from the manufacturing process. You could also consider reducing airplane trips by reducing business travel, or taking a train instead.
Normalized Retrieved Set: {'buying', 'new', 'could', 'process', 'car', 'carbon', 'answer', 'amount', 'save', 'emissions', 'used', 'public', 'favour', 'way', 'transit', 'train', 'airplane', 'correct', 'walk

In [64]:
def evaluate_generation(generated_answers, ground_truths):
    smoothing_function = SmoothingFunction().method1
    bleu_scores = []
    rouge_l_scores = []

    for generated, ground_truth in zip(generated_answers, ground_truths):
        bleu_scores.append(sentence_bleu([ground_truth.split()], generated.split(), smoothing_function=smoothing_function))
        rouge_l_scores.append(rouge_l_score(generated, ground_truth))

    return {
        "BLEU": sum(bleu_scores) / len(bleu_scores),
        "ROUGE-L": sum(rouge_l_scores) / len(rouge_l_scores)
    }

In [65]:
def evaluate_rag(retrieved_docs, generated_answers, ground_truths):
    retrieval_metrics = evaluate_retrieval(retrieved_docs, ground_truths)
    generation_metrics = evaluate_generation(generated_answers, ground_truths)

    return {**retrieval_metrics, **generation_metrics}

In [76]:
metrics = evaluate_rag(retrieved_docs, generated_answers, ground_truths)
print(metrics)

Streaming output truncated to the last 5000 lines.
Retrieved: ['The correct answer is A great way to reduce your carbon emissions is reducing the amount of car trips in favour of walking, biking, or taking public transit. Other ways could be buying items used instead of new to save on emissions from the manufacturing process. You could also consider reducing airplane trips by reducing business travel, or taking a train instead..']
Ground Truth: A great way to reduce your carbon emissions is reducing the amount of car trips in favour of walking, biking, or taking public transit. Other ways could be buying items used instead of new to save on emissions from the manufacturing process. You could also consider reducing airplane trips by reducing business travel, or taking a train instead.
Normalized Retrieved Set: {'buying', 'new', 'could', 'process', 'car', 'carbon', 'answer', 'amount', 'save', 'emissions', 'used', 'public', 'favour', 'way', 'transit', 'train', 'airplane', 'correct', 'walk